In [1]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from pandas_datareader import data
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import scipy.stats
from scipy.stats import norm
import pyfolio as pf
%matplotlib inline

/Users/marwan/opt/anaconda3/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:
import csv
df = pd.read_csv('Final_stock_data_V2.csv')
df

In [ ]:
df['sector'].hist(xrot=90)

## Filter based on 2/3 criteria

In [ ]:
def check_criteria(each):
    check = []
    if each["capm_status"] == "UnderValued":
        check.append(True)
    if each["Earnings_expectation"] == "Increase":
        check.append(True)
    if each["irv_status"] == "Below Fair Price":
        check.append(True)
    return check

In [ ]:
def check_number(criteria):
    if len(criteria) >= 2:
        return True
    else:
        return False

In [ ]:
rows = []
for i in range(938):
    each = df.iloc[i]
    criteria = check_criteria(each)
    result = check_number(criteria)
    if result:
        rows.append(i)

In [ ]:
len(rows)

In [ ]:
df = df.iloc[rows]
df

In [ ]:
#total number of stocks for each sector 

df["sector"].value_counts()

In [ ]:
#df for each sector

df_basic_materials = df.loc[df['sector'] == 'Basic Materials']
df_communication = df.loc[df['sector'] == 'Communication Services']
df_consumer_c = df.loc[df['sector'] == 'Consumer Cyclical']
df_consumer_d = df.loc[df['sector'] == 'Consumer Defensive']
df_energy = df.loc[df['sector'] == 'Energy']
df_financial = df.loc[df['sector'] == 'Financial Services']
df_healthcare = df.loc[df['sector'] == 'Healthcare']
df_industrials = df.loc[df['sector'] == 'Industrials']
df_real_estate = df.loc[df['sector'] == 'Real Estate']
df_tech = df.loc[df['sector'] == 'Technology']
df_utilites = df.loc[df['sector'] == 'Utilities']

## Selection of Sector based on Returns

In [ ]:
#calculating the average returns

basic_returns = df_basic_materials['annu_return_5y'].mean()
comm_returns = df_communication['annu_return_5y'].mean()
consumer_c_returns = df_consumer_c['annu_return_5y'].mean()
consumer_d_returns = df_consumer_d['annu_return_5y'].mean()
energy_returns = df_energy['annu_return_5y'].mean()
financial_returns = df_financial['annu_return_5y'].mean()
healthcare_returns = df_healthcare['annu_return_5y'].mean()
industrials_returns = df_industrials['annu_return_5y'].mean()
real_estate_returns = df_real_estate['annu_return_5y'].mean()
tech_returns = df_tech['annu_return_5y'].mean()
utilities_returns = df_utilites['annu_return_5y'].mean()

print(tabulate([['Basic Materials', basic_returns],['Communication Services', comm_returns],['Consumer Cyclical', consumer_c_returns],['Consumer Defensive', consumer_d_returns], ['Energy', energy_returns], ['Financial Services', financial_returns], ['Healthcare', healthcare_returns], ['Industrials', industrials_returns], ['Real Estate', real_estate_returns], ['Technology', tech_returns], ['Utilities', utilities_returns]],
               headers=['Sector','Annual Returns']))

From above, the top 3 sectors that give the highest returns is Technology (0.123625), Utilities (0.118782), Real Estate (0.106064). These will be the chosen sectors for the aggressive model. 

In [ ]:
#combining the top 3 sectors into 1 dataframe

df2 = pd.concat([df_tech, df_utilites, df_real_estate])
df2

## Filtering Stocks based on Sharpe Ratio

In [ ]:
def get_closeprice(ticker):    
    start_date = str(date.today().year - 5) +'-0'+ str(date.today().month) +'-'+ str(date.today().day)
    end_date = str(date.today())
    
    panel_data = data.DataReader([ticker],'yahoo', start_date, end_date)
    closes = panel_data[["Close", "Adj Close"]]
    
    return closes

In [ ]:
def get_sharpe_ratio(ticker):
    
    closes = get_closeprice(ticker)
    adj_close = closes["Adj Close"]
    
    risk_free_ann_ret_rate = 0.0195  #assuming this rate based on the latest data
     
    #Rp
    returns_ts = adj_close.pct_change().dropna()
    
    #Average Rp
    avg_daily_ret = returns_ts.mean()

    #Average risk-free rate for the year (Annual rate / number of trading days to spread it out)
    returns_ts['RiskFree_Rate'] = risk_free_ann_ret_rate/252

    #Average Rf
    avg_rf_ret = returns_ts['RiskFree_Rate'].mean()

    #calculate sharpe ratio

    #Add the excess return columns for each ETF: Rp - Rf
    returns_ts['Excess_ret'] = returns_ts[ticker] - returns_ts['RiskFree_Rate']
    
    sharpe = ((avg_daily_ret[ticker] - avg_rf_ret) /returns_ts['Excess_ret'].std())*np.sqrt(252)

    return sharpe.round(3)

In [ ]:
sharpe_list = []
for i in range(len(df2)):
    each = df2.iloc[i]
    ticker = each["ticker"]
    sharpe = get_sharpe_ratio(ticker)
    sharpe_list.append(sharpe)

In [ ]:
sharpe_list

In [ ]:
df2["sharpe"] = sharpe_list
df2

In [ ]:
#only choosing stocks which has a sharpe ratio of more than 0.6 

df3 = df2[df2["sharpe"] > 0.6]
df3

## Correlation between Stocks

In [ ]:
def get_return_series(ticker):
    closes = get_closeprice(ticker)
    return_series_close = (closes['Close'].pct_change()+ 1).cumprod() - 1
    return return_series_close

In [ ]:
returns_list = []
for i in range(len(df3)):
    each = df3.iloc[i]
    ticker = each["ticker"]
    returns = get_return_series(ticker)
    if (len(returns) == 1260):
        returns_list.append(returns)

In [ ]:
returns_list

In [ ]:
return_series_close = pd.concat(returns_list, axis=1)
return_series_close

In [ ]:
return_series_close.plot(figsize=(16,9))

In [ ]:
highest = return_series_close[return_series_close.tail(1) > 5]

In [ ]:
highest.dropna(axis=1, how="all")

In [ ]:
correlation = return_series_close.corr()
# correlation
fig, ax = plt.subplots(figsize=(16,10))
sns.heatmap(correlation, annot = True, ax = ax, cmap="YlGnBu")

In [ ]:
#only choosing stocks with correlation less than 1 for diversification

correlation = correlation[correlation < 0.1]
correlation

## Final DF

In [ ]:
all_stocks = []
for each in correlation:
    all_stocks.append(each)

In [ ]:
indexes = []
for i in range(len(df3)):
    each = df3.iloc[i]
    ticker = each["ticker"]
    if ticker in all_stocks:
        indexes.append(i)

In [ ]:
chosen_df = df3.iloc[indexes]
chosen_df

In [ ]:
chosen_df.to_csv("selected_stocks_aggressive.csv")

## Bonds

In [ ]:
pip install Nasdaq-Data-Link

In [ ]:
pip install quandl

In [2]:
import nasdaqdatalink
import quandl

In [3]:
quandl.get('USTREASURY/YIELD')['5 YR']
#expected return from US 5y treasury bond is 1.78

Date
1990-01-02    7.87
1990-01-03    7.92
1990-01-04    7.91
1990-01-05    7.92
1990-01-08    7.92
              ... 
2022-01-31    1.62
2022-02-01    1.63
2022-02-02    1.60
2022-02-03    1.66
2022-02-04    1.78
Name: 5 YR, Length: 8032, dtype: float64

## Aggressive Model 1: 70% Stocks & 30% Bonds

In [9]:
chosen_df = pd.read_csv('selected_stocks_aggressive.csv')
adjClose_5y = pd.read_csv('Cleaned_AdjCloseP_5y.csv', index_col='Date')

In [10]:
def calculate_returns(weights, log_returns):
    # Annual log Returns
    
    return np.sum(log_returns.mean()*weights) * 252

In [11]:
n = len(adjClose_5y[chosen_df['ticker']].columns)
log_returns_test = np.log(adjClose_5y[chosen_df['ticker']] / adjClose_5y[chosen_df['ticker']].shift(1)).dropna() # Same as 1 + stocks_data.pct_change()
log_returns_test

,ADSK,AMD,ANSS,EPAM,NTAP,QCOM,AOSL,APPS,ASML,AUDC,...,NSIT,ON,POWI,RIOT,RMBS,TTD,UCTT,WDAY,CBRE,SBAC
Date,,,,,,,,,,,,,,,,,,,,,
2017-02-06,-0.019016,0.107564,-0.001262,-0.012116,0.002088,-0.001889,-0.029289,0.000000,-0.006361,0.053571,...,-0.006116,-0.005034,-0.020928,0.031449,-0.011094,-0.004158,-0.020441,0.011427,-0.015038,-0.012280
2017-02-07,0.020082,-0.025261,0.002207,0.015768,0.012178,0.007348,-0.005769,0.014389,0.008391,-0.005979,...,-0.002671,0.006468,0.007524,0.051293,0.001592,-0.008718,0.011844,0.007934,-0.002259,0.003728
2017-02-08,-0.018755,0.020112,-0.000315,-0.013456,0.003342,-0.007159,0.004330,0.000000,0.006711,0.041122,...,-0.002946,0.003575,0.013403,-0.065323,-0.013617,-0.007383,-0.003145,-0.002910,-0.005508,0.019561
2017-02-09,0.011390,-0.010378,0.011381,0.014215,0.008944,-0.000189,0.010031,0.014185,0.003459,-0.015954,...,0.169589,-0.003575,-0.017911,0.065323,0.019961,0.024400,0.001574,0.002328,0.025342,0.007180
2017-02-10,-0.007779,0.011852,0.005591,0.010718,0.003555,0.020959,0.038690,0.027780,0.002647,0.010182,...,0.022609,0.012812,0.006006,-0.005900,0.011006,0.000344,-0.001574,-0.005597,0.074153,-0.006245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-17,-0.045290,-0.046257,-0.040391,-0.021092,-0.005257,-0.034181,-0.079797,-0.039260,-0.020438,-0.013813,...,-0.001214,-0.047373,-0.014205,-0.098070,-0.027448,-0.018713,-0.036807,-0.052144,-0.018367,-0.026176
2022-02-18,-0.029759,0.012909,-0.036769,0.005067,-0.001868,0.005023,-0.007718,-0.040005,-0.004005,-0.005005,...,0.010471,0.000166,-0.021302,-0.045502,-0.021092,-0.017275,-0.022629,-0.010390,-0.001103,-0.017587
2022-02-22,-0.005503,0.015862,0.011352,-0.035573,-0.054018,-0.010494,-0.053364,-0.055883,-0.011738,-0.001076,...,-0.004417,-0.029672,-0.021310,-0.061988,-0.016972,-0.042532,-0.036963,0.016738,-0.005230,0.021297


In [17]:
def calculate_vol(weights):
    annualized_cov = np.dot(log_returns_test.cov()*252,weights)
    vol = np.dot(weights.transpose(),annualized_cov)
    return np.sqrt(vol)
    
def calculate_returns(weights, log_returns):
    # Annual log Returns

    return np.sum(log_returns.mean()*weights) * 252


In [18]:
def function_to_minimize(weights):

    # minimize a -1 * SR (that's how the math works here, but it is the same as maximizing SR)
    return -1 * ((calculate_returns(weights, log_returns_test) - 0 )/calculate_volatility(weights, log_returns_test)) # assuming rf =0

In [19]:
required_return = 0.70 

constraints = ({'type':'eq','fun': lambda weights: np.sum(weights)-1},
               {'type':'eq','fun': lambda weights: calculate_returns(weights,log_returns_test) - required_return})
bounds = tuple((0,1) for n in range(n))
equal_weights = n * [1/n] #going to be our starting point then the function will look at either direction for minimum negative SG

In [20]:
result = minimize(fun=calculate_vol,x0=equal_weights,bounds=bounds,constraints=constraints)
result

     fun: 0.5311244845364465
     jac: array([0.20401067, 0.28892915, 0.17360228, 0.20621307, 0.12237805,
       0.16549504, 0.23905381, 0.66020308, 0.18814695, 0.15490836,
       0.19835311, 0.14229292, 0.14234179, 0.10843409, 0.12648353,
       0.13911653, 0.25966766, 0.19161016, 0.36526345, 0.16391917,
       0.49673201, 0.26036364, 0.20728758, 0.1310854 , 0.08251219])
 message: 'Optimization terminated successfully'
    nfev: 156
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([1.20248863e-16, 1.21005003e-01, 0.00000000e+00, 2.52402266e-17,
       0.00000000e+00, 1.49871186e-17, 7.87727088e-17, 5.26167600e-01,
       7.90520533e-03, 7.03172159e-02, 2.71657583e-17, 3.10678133e-17,
       7.06411925e-17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.40982987e-17, 2.01715814e-17, 6.98096321e-18,
       2.74604976e-01, 1.39832152e-17, 1.73161769e-18, 0.00000000e+00,
       3.19893851e-17])

In [21]:
min_var_weights = result['x']
min_var_weights

array([1.20248863e-16, 1.21005003e-01, 0.00000000e+00, 2.52402266e-17,
       0.00000000e+00, 1.49871186e-17, 7.87727088e-17, 5.26167600e-01,
       7.90520533e-03, 7.03172159e-02, 2.71657583e-17, 3.10678133e-17,
       7.06411925e-17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.40982987e-17, 2.01715814e-17, 6.98096321e-18,
       2.74604976e-01, 1.39832152e-17, 1.73161769e-18, 0.00000000e+00,
       3.19893851e-17])

In [26]:
result['x']
portfolio = {}
p = 0
for i in result['x']:
    print('Put',(i*100).round(3), "% in", log_returns_test.columns[p])

    portfolio[log_returns_test.columns[p]] = (i).round(5)
    p += 1

Put 0.0 % in ADSK
Put 12.101 % in AMD
Put 0.0 % in ANSS
Put 0.0 % in EPAM
Put 0.0 % in NTAP
Put 0.0 % in QCOM
Put 0.0 % in AOSL
Put 52.617 % in APPS
Put 0.791 % in ASML
Put 7.032 % in AUDC
Put 0.0 % in DIOD
Put 0.0 % in DSGX
Put 0.0 % in INOD
Put 0.0 % in MANT
Put 0.0 % in MGIC
Put 0.0 % in NSIT
Put 0.0 % in ON
Put 0.0 % in POWI
Put 0.0 % in RIOT
Put 0.0 % in RMBS
Put 27.46 % in TTD
Put 0.0 % in UCTT
Put 0.0 % in WDAY
Put 0.0 % in CBRE
Put 0.0 % in SBAC


In [27]:
cleaned_portfolio = {}
for n in portfolio:
    if portfolio[n] != 0:
        cleaned_portfolio[n] = portfolio[n]
cleaned_portfolio

{'AMD': 0.12101,
 'APPS': 0.52617,
 'ASML': 0.00791,
 'AUDC': 0.07032,
 'TTD': 0.2746}

In [28]:
list(cleaned_portfolio.keys())

['AMD', 'APPS', 'ASML', 'AUDC', 'TTD']

In [29]:
cleaned_pf_data = chosen_df[chosen_df['ticker'].isin(list(cleaned_portfolio.keys()))]
cleaned_pf_data

,Unnamed: 0,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,...,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y,sharpe
1,14,AMD,"Advanced Micro Devices, Inc.",Technology,Increase,Below Fair Price,OverValued,No Data,117.110,0.319034,...,3.242,0.3532,35.062874,36.122765,253.245268,999.0,0.256240,0.564107,0.569607,0.981
7,244,APPS,"Digital Turbine, Inc.",Technology,Increase,Below Fair Price,UnderValued,No Data,43.800,0.278657,...,0.530,0.5000,20.372091,82.641510,239.323247,999.0,0.285415,0.719036,1.325025,1.395
8,249,ASML,ASML Holding N.V. - New York Re,Technology,Increase,Below Fair Price,OverValued,No Data,634.790,0.341963,...,15.017,0.2980,32.420326,42.271423,943.608234,999.0,0.134681,0.360669,0.411064,1.065
9,258,AUDC,AudioCodes Ltd.,Technology,Increase,Below Fair Price,OverValued,No Data,27.675,0.234188,...,1.028,0.2500,16.875000,26.921206,29.306718,999.0,0.084842,0.446357,0.362672,0.818
20,890,TTD,"The Trade Desk, Inc.",Technology,Increase,Below Fair Price,OverValued,No Data,73.680,0.376786,...,0.561,0.2800,80.967030,131.336900,96.589249,999.0,0.302433,0.683392,0.956083,1.129
